In [1]:
import sys
import time
import datetime
import os
import tensorflow as tf
import argparse
import json
import socket
from PIL import Image
from util import *
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
with open("hyperParams.json") as f:
    instanceParams = json.load(f)

In [4]:
instanceParams

{u'baseLR': 1e-05,
 u'batchSize': 2,
 u'borderThicknessH': 0.1,
 u'borderThicknessW': 0.1,
 u'boundaryAlpha': 6.5,
 u'dataset': u'kitti2012',
 u'flowScale': 20.0,
 u'gradParams': {u'robustness': 0.46, u'scale': 255.0, u'weight': 6.408},
 u'instanceName': u'unsupFlownet',
 u'iterations': 500000,
 u'lossComponents': {u'asymmetricSmooth': True,
  u'backward': False,
  u'boundaries': False,
  u'gradient': False,
  u'smooth2nd': False},
 u'photoParams': {u'robustness': 0.53, u'scale': 360.0},
 u'printFreq': 50,
 u'resnet': False,
 u'smooth2ndParams': {u'robustness': 0.21, u'scale': 1.0, u'weight': 0.53},
 u'smoothOccParams': {u'robustness': 0.9, u'scale': 1.8},
 u'smoothParams': {u'robustness': 0.28, u'scale': 3.5, u'weight': 0.64},
 u'snapFreq': 5000,
 u'snapshotFreq': 10000,
 u'weightDecay': 0.0}

In [5]:
logPath = "logs/"
snapshotPath = "snapshots/"

In [6]:
printFrequency = instanceParams["printFreq"]
snapshotFrequency = instanceParams["snapFreq"]
batchSize = instanceParams["batchSize"]

iterations = instanceParams["iterations"]
baseLearningRate = instanceParams["baseLR"]
learningRate = baseLearningRate
snapshotFrequency = instanceParams["snapshotFreq"]

In [7]:
iterations = 1000

In [8]:
resume, startIteration, snapshotFiles = checkResume(snapshotPath,logPath, {'resume': 'n'})

No snapshots found, training from scratch


Below is the code used to generate train image txt files from DAVIS's train.txt.

In [9]:
# img_list = None
# with open('../example_data/data/DAVIS/ImageSets/480p/train.txt', 'r') as f:
#     img_list = f.readlines()
# img_list = [x.strip() for x in img_list]
# img_list = [item.split(' ')[0] for item in img_list]
# img_list = ["data/DAVIS" + item for item in img_list]
# img_list

# objects = sorted(['bear','car-shadow','elephant','lucia','rollerblade','blackswan','car-turn','flamingo','mallard-fly','scooter-black',
# 'bmx-bumps','cows','goat','mallard-water','scooter-gray','bmx-trees','dance-jump','hike','motocross-bumps','soapbox','boat','dance-twirl',
# 'hockey','motocross-jump','soccerball','breakdance','dog','horsejump-high','motorbike','stroller','breakdance-flare',
# 'dog-agility','horsejump-low','paragliding','surf','bus','drift-chicane','kite-surf','paragliding-launch','swing','camel',
# 'drift-straight','kite-walk','parkour','tennis','car-roundabout','drift-turn','libby','rhino','train'])

# objects = ['/' + item + '/' for item in objects]
# count = {}
# for item in objects:
#     count[item] = 0

# for im in img_list:
#     for item in objects:
#         if item in im:
#             count[item] += 1

# for key in objects:
#     try:
#         if count[key] == 0:
#             del count[key]
#     except:
#         print key


# i = 0
# with open('../example_data/datalists/train_im0.txt', 'w') as f:
#     for key in sorted(count.keys()):
#         l = count[key]
#         for j in range(l-1):
#             f.write("%s\n" % img_list[i])
#             i += 1
#         i += 1

# i = 0
# with open('../example_data/datalists/train_im1.txt', 'w') as f:
#     for key in sorted(count.keys()):
#         l = count[key]
#         i += 1
#         for j in range(l-1):
#             f.write("%s\n" % img_list[i])
#             i += 1

In [13]:
with tf.device("/gpu:0"):
    trainingData = TrainingData(batchSize,instanceParams)

Reading Tensor("ImagePairData/image_data_reader/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_1/Placeholder:0", dtype=string, device=/device:GPU:0)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
with tf.device("/gpu:0"):
    # init
    with tf.variable_scope("netShare"):
        networkBodyF = NetworkBody(trainingData,instanceParams)
    with tf.variable_scope("netShare",reuse=True):
        networkBodyB = NetworkBody(trainingData,instanceParams,flipInput=True)

    trainingLoss = TrainingLoss(instanceParams,networkBodyF,networkBodyB,trainingData)
    solver,learningRateTensor = attachSolver(trainingLoss.loss)

    # loss scheduling
    recLossBWeightTensor = trainingLoss.recLossBWeight

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [15]:
# merge summaries
merged = tf.summary.merge_all()

# saver
saver = tf.train.Saver(max_to_keep=0)

In [16]:
from dotmap import DotMap
arg = DotMap()
arg.logDev = False

In [17]:
printFrequency = 20

In [18]:
# start
with sessionSetup(arg) as sess:
#     if resume:
#         saver.restore(sess,snapshotPath+snapshotFiles[-1][:-6])
#     else:
#         sess.run(tf.initialize_all_variables())
    saver.restore(sess,
                  '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')

    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    lastPrint = time.time()
    flowFinal = networkBodyF.flows[0]
    flowViz = flowToRgb(flowFinal)
    for i in range(500000, 500000 + 20):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: learningRate,
            recLossBWeightTensor: recLossBWeight,
        }
        hi,summary,result,totalLoss = sess.run([flowViz,merged,solver,trainingLoss.loss], feed_dict=feed_dict)

        if (i+1) % printFrequency == 0:
            timeDiff = time.time() - lastPrint
            itPerSec = printFrequency/timeDiff
            remainingIt = iterations - i
            eta = remainingIt/itPerSec
            print("Iteration "+str(i+1)+": loss: "+str(totalLoss)+", iterations per second: "+str(itPerSec)+", ETA: "+str(datetime.timedelta(seconds=eta)))+", lr: "+str(learningRate)

            summary_writer.add_summary(summary,i+1)
            summary_writer.flush()
            lastPrint = time.time()

        if (i+1) % snapshotFrequency == 0:
            saver.save(sess,"snapshots/iter_"+str(i+1).zfill(16)+".ckpt")

        sys.stdout.flush()

    # close queing
#     trainingData.dataQueuer.close(sess)
    

INFO:tensorflow:Restoring parameters from ../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt
../example_data/data/DAVIS/JPEGImages/480p/dance-jump/00035.jpg
../example_data/data/DAVIS/JPEGImages/480p/dance-jump/00036.jpg
../example_data/data/DAVIS/JPEGImages/480p/horsejump-low/00007.jpg
../example_data/data/DAVIS/JPEGImages/480p/horsejump-low/00008.jpg
../example_data/data/DAVIS/JPEGImages/480p/hike/00053.jpg
../example_data/data/DAVIS/JPEGImages/480p/hike/00054.jpg
../example_data/data/DAVIS/JPEGImages/480p/scooter-gray/00012.jpg
../example_data/data/DAVIS/JPEGImages/480p/scooter-gray/00013.jpg
../example_data/data/DAVIS/JPEGImages/480p/mallard-water/00031.jpg
../example_data/data/DAVIS/JPEGImages/480p/mallard-water/00032.jpg
../example_data/data/DAVIS/JPEGImages/480p/bus/00070.jpg
../example_data/data/DAVIS/JPEGImages/480p/bus/00071.jpg
../example_data/data/DAVIS/JPEGImages/480p/flamingo/00032.jpg
../example_data/data/DAVIS/JPEGImages/480p/flamingo/00

../example_data/data/DAVIS/JPEGImages/480p/mallard-water/00024.jpg
../example_data/data/DAVIS/JPEGImages/480p/mallard-water/00025.jpg../example_data/data/DAVIS/JPEGImages/480p/elephant/00016.jpg

../example_data/data/DAVIS/JPEGImages/480p/elephant/00017.jpg
../example_data/data/DAVIS/JPEGImages/480p/motorbike/00025.jpg
../example_data/data/DAVIS/JPEGImages/480p/motorbike/00026.jpg
../example_data/data/DAVIS/JPEGImages/480p/paragliding/00012.jpg
../example_data/data/DAVIS/JPEGImages/480p/paragliding/00013.jpg
../example_data/data/DAVIS/JPEGImages/480p/mallard-fly/00064.jpg
../example_data/data/DAVIS/JPEGImages/480p/mallard-fly/00065.jpg
../example_data/data/DAVIS/JPEGImages/480p/breakdance-flare/00029.jpg
../example_data/data/DAVIS/JPEGImages/480p/breakdance-flare/00030.jpg
../example_data/data/DAVIS/JPEGImages/480p/stroller/00048.jpg../example_data/data/DAVIS/JPEGImages/480p/drift-turn/00037.jpg
../example_data/data/DAVIS/JPEGImages/480p/drift-turn/00038.jpg

../example_data/data/DAVIS